In [3]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd

In [4]:
def get_jobs(keyword, num_jobs, verbose):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path='C:\\Users\\kiit\\Documents\\machine learning\\web_scraping\\chromedriver.exe', options=options)
    driver.set_window_size(1120, 1000)
    
    url = 'https://www.glassdoor.co.in/Job/'+keyword+'-jobs-SRCH_KO0,32.htm'
#     url = "https://www.glassdoor.co.in/Job/"+keyword+"-jobs-SRCH_KO0,25.htm"
#     url = "https://www.glassdoor.co.in/Job/"+keyword+"-jobs-SRCH_KO0,14.htm"
#     url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(3)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try: 
            driver.find_element_by_xpath('//*[@id="prefix__icon-close-1"]').click()  #clicking to the X.
        except NoSuchElementException:
            pass

        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            #print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            job_button.click()  #You might 
            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(2)

            try:
                salary_estimate = driver.find_element_by_xpath('.//span[@class="gray small salary"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break
            

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.


In [5]:
df = get_jobs("artificial-intelligence-engineer", 300, False)
df.to_csv("glassdoor_artificial-intelligence-engineer300.csv",index=False)

In [81]:
machine-learning-engineer = 1,539
artificial-intelligence-engineer  = 370

,Company Name,Competitors,Founded,Headquarters,Industry,Job Description,Job Title,Location,Rating,Revenue,Salary Estimate,Sector,Size,Type of ownership
0,IBM\n3.9,"Amazon, Accenture, Microsoft",1911,"Armonk, NY",IT Services,Introduction\nSoftware Developers at IBM are t...,Data Scientist,Bengaluru,3.9,₹500+ billion (INR),-1,Information Technology,10000+ employees,Company - Public
1,Quanticate\n4.0,"GCE Solutions, Parexel, IQVIA",1995,"Hitchin, United Kingdom",Biotech & Pharmaceuticals,Overview\n\n\nWe have an exciting opportunity ...,Data Scientist,Bengaluru,4.0,₹1 to ₹5 billion (INR),-1,Biotech & Pharmaceuticals,201 to 500 employees,Company - Private
2,Amazon\n4.2,"Google, Microsoft, Walmart",1994,"Seattle, WA",Internet,"At Amazon, we strive to be most customer-centr...",Data Scientist,Hyderabad,4.2,₹500+ billion (INR),-1,Information Technology,10000+ employees,Company - Public
3,PayPal\n3.5,"Square, Amazon, Apple",1998,"San Jose, CA",Internet,Provide analytical insights into emerging prob...,Data Scientist,Bengaluru,3.5,₹500+ billion (INR),-1,Information Technology,10000+ employees,Company - Public
4,HP Inc.\n4.0,-1,1939,"Palo Alto, CA",Computer Hardware & Software,HP is the worlds leading personal systems and ...,Data Scientist,Bengaluru,4.0,Unknown / Non-Applicable,-1,Information Technology,10000+ employees,Company - Public


In [2]:
counter = 0
for i in range(1501):
    if 
    counter+=1
    print(counter)
    print(i)

1
0
2
1
3
2
4
3
5
4
6
5
7
6
8
7
9
8
10
9
11
10
12
11
13
12
14
13
15
14
16
15
17
16
18
17
19
18
20
19
21
20
22
21
23
22
24
23
25
24
26
25
27
26
28
27
29
28
30
29
31
30
32
31
33
32
34
33
35
34
36
35
37
36
38
37
39
38
40
39
41
40
42
41
43
42
44
43
45
44
46
45
47
46
48
47
49
48
50
49
51
50
52
51
53
52
54
53
55
54
56
55
57
56
58
57
59
58
60
59
61
60
62
61
63
62
64
63
65
64
66
65
67
66
68
67
69
68
70
69
71
70
72
71
73
72
74
73
75
74
76
75
77
76
78
77
79
78
80
79
81
80
82
81
83
82
84
83
85
84
86
85
87
86
88
87
89
88
90
89
91
90
92
91
93
92
94
93
95
94
96
95
97
96
98
97
99
98
100
99
101
100
102
101
103
102
104
103
105
104
106
105
107
106
108
107
109
108
110
109
111
110
112
111
113
112
114
113
115
114
116
115
117
116
118
117
119
118
120
119
121
120
122
121
123
122
124
123
125
124
126
125
127
126
128
127
129
128
130
129
131
130
132
131
133
132
134
133
135
134
136
135
137
136
138
137
139
138
140
139
141
140
142
141
143
142
144
143
145
144
146
145
147
146
148
147
149
148
150
149
151
150
152
151
15

In [74]:
# import requests
# import bs4
# from bs4 import BeautifulSoup
# import pandas as pd


# url = "https://www.indeed.com/jobs"
# headers = {'User-Agent': "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.134 Safari/537.36"}
# title = 'datascientist'
# location = 'india'

# params = {'q': title, 'l':location,
#         'start':1
#         }


# r = requests.get(url=url,headers=headers ,params=params)

# r.text